In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
!pip install -U langchain langchain-community sentence-transformers transformers accelerate faiss-cpu pypdf streamlit langchain-huggingface langchain-core


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 42.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


In [ ]:
# !pip uninstall -y transformers sentence-transformers langchain langchain-community langchain-huggingface
# !pip cache purge


In [ ]:
# !pip install transformers==4.38.2 sentence-transformers==2.6.1 langchain==0.1.16 langchain-community==0.0.36  langchain-huggingface==0.0.3 accelerate faiss-cpu


In [ ]:
# !pip uninstall -y transformers sentence-transformers
# !pip install -U transformers==4.38.2 sentence-transformers==2.6.1 accelerate


In [5]:
from transformers import pipeline


2025-12-23 19:39:05.601784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766518745.660186     337 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766518745.677876     337 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
from transformers import AutoModel
# from sentence_transformers import SentenceTransformer

SentenceTransformer("intfloat/e5-small-v2")
print("OK: embeddings load")


In [7]:
# import os
import requests

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline

In [ ]:
# from sentence_transformers import SentenceTransformer


In [9]:
PDF_URL = "https://www.fema.gov/sites/default/files/documents/fema_nfip_flood-insurance-manual_102025.pdf"
PDF_PATH = "nfip_manual.pdf"

In [11]:
def download_pdf():
    if not os.path.exists(PDF_PATH):
        response = requests.get(PDF_URL)
        with open(PDF_PATH, "wb") as f:
            f.write(response.content)

In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)[:1500]

In [13]:
def build_rag_chain():
    # Download PDF
    download_pdf()

    # Load PDF
    loader = PyPDFLoader(PDF_PATH)
    pages = loader.load()

    # Chunk
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n\n", "\n", ".", " "]
    )
    docs = splitter.split_documents(pages)

    # Embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/e5-small-v2",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

    vectorstore = FAISS.from_documents(docs, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # LLM
    hf_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-base",
        max_new_tokens=256
    )
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    # Prompt
    prompt = PromptTemplate.from_template("""
    You are an assistant answering questions about FEMA's NFIP Flood Insurance Manual.

    Use ONLY the provided context.
    List ALL applicable items.
    Do NOT summarize or omit items.

    Context:
    {context}

    Question:
    {question}

    

    Answer (bulleted list):
    """)

    rag_chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )

    return rag_chain


In [14]:
# Build once at import time
RAG_CHAIN = build_rag_chain()

/tmp/ipykernel_337/2693083099.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [15]:
def ask_question(question: str) -> str:
    """Call this from Streamlit"""
    return RAG_CHAIN.invoke(question)


In [26]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Heading
display(Markdown("## 🌊 Flood Insurance Chatbot"))
display(Markdown("_Ask questions about FEMA's NFIP Flood Insurance Manual_"))

# Input box
textbox = widgets.Text(
    placeholder="Ask a question about flood insurance...",
    layout=widgets.Layout(width="70%")
)

# Buttons
ask_button = widgets.Button(description="Ask", button_style="primary")
clear_button = widgets.Button(description="Clear Chat", button_style="warning")

# Output area (chat history)
output = widgets.Output()

# Handler for sending question
def handle_ask(_):
    question = textbox.value.strip()
    if not question:
        return
    
    with output:
        display(Markdown(f"**🧑 You:** {question}"))
        try:
            # Call your RAG chain
            # answer = rag_chain.invoke(question)
            display(Markdown(f"**🤖 Bot:** {ask_question(question)}"))
        except Exception as e:
            display(Markdown(f"❌ Error: {e}"))
    
    textbox.value = ""  # clear input

# Handler for clearing chat
def handle_clear(_):
    output.clear_output()

# Attach buttons
ask_button.on_click(handle_ask)
clear_button.on_click(handle_clear)

# Layout
display(widgets.HBox([textbox, ask_button, clear_button]), output)


## 🌊 Flood Insurance Chatbot

_Ask questions about FEMA's NFIP Flood Insurance Manual_

Output()